# Symbolic nonlinear observability for monocular camera example

In [1]:
import numpy as np
import sympy as sp

In [2]:
# Import functions directly from github
# Important: note that we use raw.githubusercontent.com, not github.com

import requests
url = 'https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Utility/symbolic_derivatives.py'
r = requests.get(url)

# Store the file to the colab working directory
with open('symbolic_derivatives.py', 'w') as f:
    f.write(r.text)

# import the function we want from that file
import symbolic_derivatives

# Define states

In [14]:
g, d = sp.symbols(['g', 'd'])
x = [g, d]

# Define dynamics

In [15]:
f_0 = sp.Matrix([0,
                 0])
f_1 = sp.Matrix([1,
                 0])

# Define measurements

In [16]:
h = sp.Matrix([g/d])

# Calculate each term in G

In [17]:
L_f0_h = symbolic_derivatives.directional_derivative(h, x, f_0)
L_f1_h = symbolic_derivatives.directional_derivative(h, x, f_1)
#L_f1_f1_h = symbolic_derivatives.directional_derivative(L_f1_h, x, f_1) #floris needs to verify this

In [18]:
L_f1_h

Matrix([[1/d]])

# Assemble G, take Jacobian

In [19]:
G = sp.Matrix([h, L_f0_h, L_f1_h])

In [20]:
G

Matrix([
[g/d],
[  0],
[1/d]])

In [21]:
G.jacobian(x) #this is equivalent to the observability matrix in linear case

Matrix([
[1/d, -g/d**2],
[  0,       0],
[  0, -1/d**2]])

# Plug in x0

In [22]:
x0 = {g: 1, d: 2}

In [23]:
G.jacobian(x).subs(x0)

Matrix([
[1/2, -1/4],
[  0,    0],
[  0, -1/4]])

In [24]:
G.jacobian(x).subs(x0).rank() # rank of non square nxm matrix is min{n,m} so this is full rank

2

# Shortcut function to get G:

### First derivatives

In [26]:
G1 =  symbolic_derivatives.get_bigO( h,
                                    x,
                                    [f_0,
                                     f_1],
                                   )
G1

[Matrix([[g/d]]), Matrix([[0]]), Matrix([[1/d]])]

### Second derivatives

In [27]:
G2 =  symbolic_derivatives.get_bigO( sp.Matrix.vstack(*G1),
                                    x,
                                    [f_0,
                                     f_1],
                                   )
G2

[Matrix([
 [g/d],
 [  0],
 [1/d]]),
 Matrix([
 [0],
 [0],
 [0]]),
 Matrix([
 [1/d],
 [  0],
 [  0]])]

In [28]:
G = sp.Matrix.vstack(*G1, *G2)
G

Matrix([
[g/d],
[  0],
[1/d],
[g/d],
[  0],
[1/d],
[  0],
[  0],
[  0],
[1/d],
[  0],
[  0]])